In [ ]:
from typing import List, Optional, Dict
from langchain_core.load.serializable import Serializable
from pydantic import BaseModel, Field
# from langchain_neo4j.graphs.graph_document import Relationship, Node

from src.config import LLMConf
from src.schema import Ontology
from src.factory.llm import fetch_llm
from src.prompts.graph_extractor import get_graph_extractor_prompt

In [ ]:
llm_conf = LLMConf(
    type="ollama",
    model="llama3.2:latest", 
    temperature=0.0, 
)

llm = fetch_llm(llm_conf)

class Node(Serializable):
    id: str
    type: str
    properties: Optional[Dict[str, str]] = None


class Relationship(Serializable):
    source: str
    target: str
    type: str
    properties: Optional[Dict[str, str]] = None


class Graph(Serializable):
    """ 
    Represents a graph consisting of nodes and relationships.

    Attributes:
        nodes (List[Node]): A list of nodes in the graph.
        relationships (List[Relationship]): A list of relationships in the graph.
    """
    nodes: List[Node]
    relationships: List[Relationship]

prompt = get_graph_extractor_prompt()

In [ ]:
llm

In [ ]:
print(prompt.template)

In [ ]:
ontology = Ontology(allowed_labels=["Person", "City", "Country", "Region"], allowed_relations=["LIVES_IN", "BORN_IN"])

prompt.partial_variables = {
    'allowed_labels':ontology.allowed_labels if ontology.allowed_labels else "", 
    'labels_descriptions': ontology.labels_descriptions if ontology.labels_descriptions else "", 
    'allowed_relationships': ontology.allowed_relations if ontology.allowed_relations else ""
}


In [ ]:
text= """ 
    Meet Marco Rossi, a proud native of Rome, Italy. With a passion for pasta and a love for football, 
    Marco embodies the vibrant spirit of his homeland. 
    Growing up amidst the ancient ruins and bustling piazzas of Rome, Marco developed 
    a deep appreciation for his city's rich history and cultural heritage. 
    From the iconic Colosseum to the majestic Vatican City, he has explored every corner of the Eternal City, 
    finding beauty in its timeless landmarks and hidden gems. 
    Italy, with its delectable cuisine and stunning landscapes, has always been Marco's playground. 
    From the rolling hills of Tuscany to the sparkling waters of the Amalfi Coast,
    he has embraced the diversity and beauty of his country, taking pride in its traditions and way of life.
"""

print(prompt.format(input_text=text))

In [ ]:
output = llm.with_structured_output(schema=Graph).invoke(input=prompt.format(input_text=text))

In [ ]:
output

In [ ]:
output.relationships

In [ ]:
output.nodes